<a href="https://colab.research.google.com/github/tiffanysn/general_learning/blob/dev/Quantium_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [346]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load required libraries and datasets

In [347]:
! cp drive/My\ Drive/QVI_data.csv .

In [348]:
import pandas as pd

In [349]:
import plotly.express as px

In [350]:
import numpy as np


In [351]:
df=pd.read_csv('QVI_data.csv')

In [352]:
df.shape

(264834, 12)

In [353]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LYLTY_CARD_NBR    264834 non-null  int64  
 1   DATE              264834 non-null  object 
 2   STORE_NBR         264834 non-null  int64  
 3   TXN_ID            264834 non-null  int64  
 4   PROD_NBR          264834 non-null  int64  
 5   PROD_NAME         264834 non-null  object 
 6   PROD_QTY          264834 non-null  int64  
 7   TOT_SALES         264834 non-null  float64
 8   PACK_SIZE         264834 non-null  int64  
 9   BRAND             264834 non-null  object 
 10  LIFESTAGE         264834 non-null  object 
 11  PREMIUM_CUSTOMER  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


In [354]:
df.describe(include= 'all')

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
count,2.648340e+05,264834,264834.000000,2.648340e+05,264834.000000,264834,264834.000000,264834.000000,264834.000000,264834,264834,264834
unique,NaN,364,NaN,NaN,NaN,114,NaN,NaN,NaN,21,7,3
top,NaN,2018-12-24,NaN,NaN,NaN,Kettle Mozzarella Basil & Pesto 175g,NaN,NaN,NaN,KETTLE,OLDER SINGLES/COUPLES,Mainstream
freq,NaN,939,NaN,NaN,NaN,3304,NaN,NaN,NaN,41288,54479,101988
mean,1.355488e+05,NaN,135.079423,1.351576e+05,56.583554,NaN,1.905813,7.299346,182.425512,NaN,NaN,NaN
std,8.057990e+04,NaN,76.784063,7.813292e+04,32.826444,NaN,0.343436,2.527241,64.325148,NaN,NaN,NaN
min,1.000000e+03,NaN,1.000000,1.000000e+00,1.000000,NaN,1.000000,1.500000,70.000000,NaN,NaN,NaN
25%,7.002100e+04,NaN,70.000000,6.760050e+04,28.000000,NaN,2.000000,5.400000,150.000000,NaN,NaN,NaN
50%,1.303570e+05,NaN,130.000000,1.351365e+05,56.000000,NaN,2.000000,7.400000,170.000000,NaN,NaN,NaN
75%,2.030940e+05,NaN,203.000000,2.026998e+05,85.000000,NaN,2.000000,9.200000,175.000000,NaN,NaN,NaN


In [355]:
df.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


## Select control store

#### Add Month column

In [356]:
import datetime

In [357]:
df['year'] = pd.DatetimeIndex(df['DATE']).year
df['month']=pd.DatetimeIndex(df['DATE']).month
df['year_month']=pd.to_datetime(df['DATE']).dt.floor('d') - pd.offsets.MonthBegin(1)
df

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,year,month,year_month
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,2018,10,2018-10-01
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,2018,9,2018-09-01
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,2019,3,2019-03-01
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,2019,3,2019-03-01
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,2018,11,2018-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264829,2370701,2018-12-08,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,210,GRNWVES,YOUNG FAMILIES,Mainstream,2018,12,2018-12-01
264830,2370751,2018-10-01,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,KETTLE,YOUNG FAMILIES,Premium,2018,10,2018-09-01
264831,2370961,2018-10-24,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,165,TYRRELLS,OLDER FAMILIES,Budget,2018,10,2018-10-01
264832,2370961,2018-10-27,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,300,OLD,OLDER FAMILIES,Budget,2018,10,2018-10-01


#### Monthly calculation for each store

In [358]:
totSales= df.groupby(['STORE_NBR','year_month'])['TOT_SALES'].sum().reset_index()
totSales

,STORE_NBR,year_month,TOT_SALES
0,1,2018-06-01,4.8
1,1,2018-07-01,220.2
2,1,2018-08-01,171.8
3,1,2018-09-01,278.2
4,1,2018-10-01,186.4
...,...,...,...
3382,272,2019-02-01,395.5
3383,272,2019-03-01,478.5
3384,272,2019-04-01,415.5
3385,272,2019-05-01,322.8


In [359]:
measureOverTime2 = pd.DataFrame(data=totSales)

In [360]:
nTxn= df.groupby(['STORE_NBR','year_month'])['TXN_ID'].count().reset_index(drop=True)
nTxn

0        2
1       55
2       41
3       62
4       45
        ..
3382    48
3383    58
3384    52
3385    41
3386    35
Name: TXN_ID, Length: 3387, dtype: int64

In [361]:
sorted(df['year_month'].unique())

[numpy.datetime64('2018-06-01T00:00:00.000000000'),
 numpy.datetime64('2018-07-01T00:00:00.000000000'),
 numpy.datetime64('2018-08-01T00:00:00.000000000'),
 numpy.datetime64('2018-09-01T00:00:00.000000000'),
 numpy.datetime64('2018-10-01T00:00:00.000000000'),
 numpy.datetime64('2018-11-01T00:00:00.000000000'),
 numpy.datetime64('2018-12-01T00:00:00.000000000'),
 numpy.datetime64('2019-01-01T00:00:00.000000000'),
 numpy.datetime64('2019-02-01T00:00:00.000000000'),
 numpy.datetime64('2019-03-01T00:00:00.000000000'),
 numpy.datetime64('2019-04-01T00:00:00.000000000'),
 numpy.datetime64('2019-05-01T00:00:00.000000000'),
 numpy.datetime64('2019-06-01T00:00:00.000000000')]

In [362]:
measureOverTime2['nCustomers'] = df.groupby(['STORE_NBR','year_month','LYLTY_CARD_NBR'])['DATE'].count().groupby(['STORE_NBR','year_month']).count().reset_index(drop=True)
measureOverTime2.head()

,STORE_NBR,year_month,TOT_SALES,nCustomers
0,1,2018-06-01,4.8,2
1,1,2018-07-01,220.2,52
2,1,2018-08-01,171.8,41
3,1,2018-09-01,278.2,59
4,1,2018-10-01,186.4,44


In [363]:
measureOverTime2['nTxnPerCust'] = nTxn/measureOverTime2['nCustomers']
measureOverTime2.head()

,STORE_NBR,year_month,TOT_SALES,nCustomers,nTxnPerCust
0,1,2018-06-01,4.8,2,1.000000
1,1,2018-07-01,220.2,52,1.057692
2,1,2018-08-01,171.8,41,1.000000
3,1,2018-09-01,278.2,59,1.050847
4,1,2018-10-01,186.4,44,1.022727


In [364]:
totQty = df.groupby(['STORE_NBR','year_month'])['PROD_QTY'].sum().reset_index(drop=True)
totQty

0         2
1        65
2        53
3        75
4        57
       ... 
3382     91
3383    111
3384     97
3385     73
3386     66
Name: PROD_QTY, Length: 3387, dtype: int64

In [365]:
measureOverTime2['nChipsPerTxn'] = totQty/nTxn
measureOverTime2

,STORE_NBR,year_month,TOT_SALES,nCustomers,nTxnPerCust,nChipsPerTxn
0,1,2018-06-01,4.8,2,1.000000,1.000000
1,1,2018-07-01,220.2,52,1.057692,1.181818
2,1,2018-08-01,171.8,41,1.000000,1.292683
3,1,2018-09-01,278.2,59,1.050847,1.209677
4,1,2018-10-01,186.4,44,1.022727,1.266667
...,...,...,...,...,...,...
3382,272,2019-02-01,395.5,45,1.066667,1.895833
3383,272,2019-03-01,478.5,52,1.115385,1.913793
3384,272,2019-04-01,415.5,50,1.040000,1.865385
3385,272,2019-05-01,322.8,36,1.138889,1.780488


In [366]:
measureOverTime2['avgPricePerUnit'] = totSales['TOT_SALES']/totQty
measureOverTime2

,STORE_NBR,year_month,TOT_SALES,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,1,2018-06-01,4.8,2,1.000000,1.000000,2.400000
1,1,2018-07-01,220.2,52,1.057692,1.181818,3.387692
2,1,2018-08-01,171.8,41,1.000000,1.292683,3.241509
3,1,2018-09-01,278.2,59,1.050847,1.209677,3.709333
4,1,2018-10-01,186.4,44,1.022727,1.266667,3.270175
...,...,...,...,...,...,...,...
3382,272,2019-02-01,395.5,45,1.066667,1.895833,4.346154
3383,272,2019-03-01,478.5,52,1.115385,1.913793,4.310811
3384,272,2019-04-01,415.5,50,1.040000,1.865385,4.283505
3385,272,2019-05-01,322.8,36,1.138889,1.780488,4.421918


#### Filter pre-trial & stores with full obs

In [367]:
measureOverTime2.set_index('year_month', inplace=True)

In [368]:
preTrialMeasures = measureOverTime2.loc['2018-06-01':'2019-01-01'].reset_index()
preTrialMeasures

,year_month,STORE_NBR,TOT_SALES,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,2018-06-01,1,4.8,2,1.000000,1.000000,2.400000
1,2018-07-01,1,220.2,52,1.057692,1.181818,3.387692
2,2018-08-01,1,171.8,41,1.000000,1.292683,3.241509
3,2018-09-01,1,278.2,59,1.050847,1.209677,3.709333
4,2018-10-01,1,186.4,44,1.022727,1.266667,3.270175
...,...,...,...,...,...,...,...
2062,2018-09-01,272,295.9,31,1.129032,1.971429,4.288406
2063,2018-10-01,272,438.0,45,1.155556,1.942308,4.336634
2064,2018-11-01,272,384.0,42,1.095238,1.934783,4.314607
2065,2018-12-01,272,388.7,45,1.000000,1.888889,4.572941


#### Owen's *Solution*

In [369]:
measureOverTime = df.groupby(['STORE_NBR','year_month','LYLTY_CARD_NBR']).\
                      agg(
                          totSalesPerCust=('TOT_SALES', sum),
                          nTxn=('TXN_ID', "count"),
                          nChips=('PROD_QTY', sum)
                          ).\
                      groupby(['STORE_NBR','year_month']).\
                      agg(
                          totSales=("totSalesPerCust", sum),
                          nCustomers=("nTxn", "count"),
                          nTxnPerCust=("nTxn", lambda x: x.sum()/x.count()),
                          totChips=("nChips", sum),
                          totTxn=("nTxn", sum)).\
                      reset_index()

In [370]:
measureOverTime['nChipsPerTxn'] = measureOverTime['totChips']/measureOverTime['totTxn']
measureOverTime['avgPricePerUnit'] = measureOverTime['totSales']/measureOverTime['totChips']
measureOverTime.drop(['totChips', 'totTxn'], axis=1, inplace=True)

#### Calculate correlation

In [371]:
preTrialMeasures

,year_month,STORE_NBR,TOT_SALES,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,2018-06-01,1,4.8,2,1.000000,1.000000,2.400000
1,2018-07-01,1,220.2,52,1.057692,1.181818,3.387692
2,2018-08-01,1,171.8,41,1.000000,1.292683,3.241509
3,2018-09-01,1,278.2,59,1.050847,1.209677,3.709333
4,2018-10-01,1,186.4,44,1.022727,1.266667,3.270175
...,...,...,...,...,...,...,...
2062,2018-09-01,272,295.9,31,1.129032,1.971429,4.288406
2063,2018-10-01,272,438.0,45,1.155556,1.942308,4.336634
2064,2018-11-01,272,384.0,42,1.095238,1.934783,4.314607
2065,2018-12-01,272,388.7,45,1.000000,1.888889,4.572941


In [372]:
# Input
inputTable = preTrialMeasures
metricCol = 'TOT_SALES'
storeComparison = 77

x = 1

In [373]:
corr = preTrialMeasures.\
          loc[preTrialMeasures['STORE_NBR'].\
          isin([x,storeComparison])].\
          loc[:, ['year_month', 'STORE_NBR', metricCol]].\
          pivot(index='year_month', columns='STORE_NBR', values=metricCol).\
          corr().\
          iloc[0, 1]

In [374]:
preTrialMeasures.loc[preTrialMeasures['STORE_NBR'].isin([x,storeComparison])].loc[:, ['year_month', 'STORE_NBR', metricCol]].\
pivot(index='year_month', columns='STORE_NBR', values=metricCol).corr()
          

STORE_NBR,1,77
STORE_NBR,,
1,1.000000,0.841751
77,0.841751,1.000000


In [375]:
df = pd.DataFrame(columns=['Store1', 'Store2', 'corr_measure'])

In [376]:
df.append({'Store1':x, 'Store2':storeComparison, 'corr_measure':corr}, ignore_index=True)

,Store1,Store2,corr_measure
0,1.0,77.0,0.841751


In [377]:
def calculateCorrelation(inputTable, metricCol, storeComparison):
    df = pd.DataFrame(columns=['Store1', 'Store2', 'corr_measure'])
    for x in inputTable.STORE_NBR.unique():
      if x in [77, 86, 88]:
        pass
      else:
        corr = inputTable.\
                        loc[inputTable['STORE_NBR'].\
                        isin([x,storeComparison])].\
                        loc[:, ['year_month', 'STORE_NBR', metricCol]].\
                        pivot(index='year_month', columns='STORE_NBR', values=metricCol).\
                        corr().\
                        iloc[0, 1]
        df = df.append({'Store1':storeComparison, 'Store2':x, 'corr_measure':corr}, ignore_index=True)
    return(df)

In [378]:
calcCorrTable = calculateCorrelation(inputTable=preTrialMeasures, metricCol='nCustomers', storeComparison=77)

In [379]:
calcCorrTable

,Store1,Store2,corr_measure
0,77.0,1.0,0.909962
1,77.0,2.0,0.881730
2,77.0,3.0,0.975036
3,77.0,4.0,0.916797
4,77.0,5.0,0.962518
...,...,...,...
263,77.0,268.0,0.913429
264,77.0,269.0,0.925431
265,77.0,270.0,0.895506
266,77.0,271.0,0.943397


#### Calculate magnitude distance

In [380]:
inputTable = preTrialMeasures
metricCol = 'TOT_SALES'
storeComparison = '77'

x='2'

In [381]:
mag = preTrialMeasures.\
          loc[preTrialMeasures['STORE_NBR'].isin([x, storeComparison])].\
          loc[:, ['year_month', 'STORE_NBR', metricCol]].\
          pivot(index='year_month', columns='STORE_NBR', values=metricCol).\
          reset_index().rename_axis(None, axis=1)
mag

,year_month,2,77
0,2018-06-01,12.1,15.6
1,2018-07-01,145.2,289.1
2,2018-08-01,191.9,247.6
3,2018-09-01,152.8,225.2
4,2018-10-01,170.1,204.5
5,2018-11-01,163.5,247.7
6,2018-12-01,133.1,270.1
7,2019-01-01,166.1,210.2


In [382]:
mag.columns = mag.columns.map(str)
mag

,year_month,2,77
0,2018-06-01,12.1,15.6
1,2018-07-01,145.2,289.1
2,2018-08-01,191.9,247.6
3,2018-09-01,152.8,225.2
4,2018-10-01,170.1,204.5
5,2018-11-01,163.5,247.7
6,2018-12-01,133.1,270.1
7,2019-01-01,166.1,210.2


In [383]:
mag['measures'] = mag.apply(lambda row: row[x]-row[storeComparison], axis=1).abs()

mag

,year_month,2,77,measures
0,2018-06-01,12.1,15.6,3.5
1,2018-07-01,145.2,289.1,143.9
2,2018-08-01,191.9,247.6,55.7
3,2018-09-01,152.8,225.2,72.4
4,2018-10-01,170.1,204.5,34.4
5,2018-11-01,163.5,247.7,84.2
6,2018-12-01,133.1,270.1,137.0
7,2019-01-01,166.1,210.2,44.1


In [384]:
mag['Store1'] = x
mag['Store2'] = storeComparison

In [385]:
df_temp = mag.loc[:, ['Store1', 'Store2', 'year_month','measures']]

In [386]:
df_temp

,Store1,Store2,year_month,measures
0,2,77,2018-06-01,3.5
1,2,77,2018-07-01,143.9
2,2,77,2018-08-01,55.7
3,2,77,2018-09-01,72.4
4,2,77,2018-10-01,34.4
5,2,77,2018-11-01,84.2
6,2,77,2018-12-01,137.0
7,2,77,2019-01-01,44.1


In [387]:
df = pd.DataFrame(columns=['Store1', 'Store2', 'year_month','measures'])
df

,Store1,Store2,year_month,measures


In [388]:
inputTable = preTrialMeasures
metricCol = 'TOT_SALES'
storeComparison = '77'
df = pd.DataFrame(columns=['Store1', 'Store2', 'year_month','measures'])
for x in inputTable.STORE_NBR.unique():
      if x in [77, 86, 88]:
        pass
      else:
        mag = preTrialMeasures.\
          loc[preTrialMeasures['STORE_NBR'].\
          isin([x, storeComparison])].\
          loc[:, ['year_month', 'STORE_NBR', metricCol]].\
          pivot(index='year_month', columns='STORE_NBR', values=metricCol).\
          reset_index().rename_axis(None, axis=1)
        mag.columns = ['year_month', 'Store1', 'Store2']
        mag['measures'] = mag.apply(lambda row: row['Store1']-row['Store2'], axis=1).abs()
        mag['Store1'] = x
        mag['Store2'] = storeComparison 
        df_temp = mag.loc[:, ['Store1', 'Store2', 'year_month','measures']]
        df = pd.concat([df, df_temp])

In [389]:
df

,Store1,Store2,year_month,measures
0,1,77,2018-06-01,10.8
1,1,77,2018-07-01,68.9
2,1,77,2018-08-01,75.8
3,1,77,2018-09-01,53.0
4,1,77,2018-10-01,18.1
...,...,...,...,...
3,272,77,2018-09-01,70.7
4,272,77,2018-10-01,233.5
5,272,77,2018-11-01,136.3
6,272,77,2018-12-01,118.6


In [390]:
def calculateMagnitudeDistance(inputTable, metricCol, storeComparison):
  df = pd.DataFrame(columns=['Store1', 'Store2', 'year_month','measures'])
  for x in inputTable.STORE_NBR.unique():
      if x in [77, 86, 88]:
        pass
      else:
        mag = preTrialMeasures.\
          loc[preTrialMeasures['STORE_NBR'].\
          isin([x, storeComparison])].\
          loc[:, ['year_month', 'STORE_NBR', metricCol]].\
          pivot(index='year_month', columns='STORE_NBR', values=metricCol).\
          reset_index().rename_axis(None, axis=1)
        mag.columns = ['year_month', 'Store1', 'Store2']
        mag['measures'] = mag.apply(lambda row: row['Store1']-row['Store2'], axis=1).abs()
        mag['Store1'] = storeComparison
        mag['Store2'] = x 
        df_temp = mag.loc[:, ['Store1', 'Store2', 'year_month','measures']]
        df = pd.concat([df, df_temp])
  return df

In [403]:
def finalDistTable(inputTable, metricCol, storeComparison):
      calcDistTable = calculateMagnitudeDistance(inputTable, metricCol, storeComparison)
      minMaxDist = calcDistTable.groupby(['Store1','year_month'])['measures'].agg(['max','min']).reset_index()
      distTable = calcDistTable.merge(minMaxDist, on=['year_month', 'Store1'])
      distTable['magnitudeMeasure']= distTable.apply(lambda row: 1- (row['measures']-row['min'])/(row['max']-row['min']),axis=1)
      finalDistTable = distTable.groupby(['Store1','Store2'])['magnitudeMeasure'].mean().reset_index()
      finalDistTable.columns = ['Store1','Store2','mag_measure']
      return finalDistTable

In [392]:
calcDistTable = calculateMagnitudeDistance(inputTable=preTrialMeasures, metricCol='nCustomers', storeComparison='77')
calcDistTable

,Store1,Store2,year_month,measures
0,77,1,2018-06-01,0
1,77,1,2018-07-01,2
2,77,1,2018-08-01,4
3,77,1,2018-09-01,17
4,77,1,2018-10-01,7
...,...,...,...,...
3,77,272,2018-09-01,11
4,77,272,2018-10-01,8
5,77,272,2018-11-01,0
6,77,272,2018-12-01,1


#### Standardise the magnitude distance

In [393]:
#calcDistTable.groupby(['Store1','year_month'])['measures'].apply(lambda g: g.max() - g.min()).reset_index()

In [394]:
minMaxDist = calcDistTable.groupby(['Store1','year_month'])['measures'].agg(['max','min']).reset_index()
minMaxDist

,Store1,year_month,max,min
0,77,2018-06-01,8,0.0
1,77,2018-07-01,91,0.0
2,77,2018-08-01,92,0.0
3,77,2018-09-01,96,0.0
4,77,2018-10-01,102,0.0
5,77,2018-11-01,98,0.0
6,77,2018-12-01,105,0.0
7,77,2019-01-01,100,0.0


In [395]:
calcDistTable.merge(minMaxDist, on=['year_month', 'Store1'])

,Store1,Store2,year_month,measures,max,min
0,77,1,2018-06-01,0,8,0.0
1,77,2,2018-06-01,0,8,0.0
2,77,3,2018-06-01,1,8,0.0
3,77,4,2018-06-01,3,8,0.0
4,77,5,2018-06-01,0,8,0.0
...,...,...,...,...,...,...
2139,77,268,2019-01-01,1,100,0.0
2140,77,269,2019-01-01,71,100,0.0
2141,77,270,2019-01-01,82,100,0.0
2142,77,271,2019-01-01,57,100,0.0


In [396]:
distTable = calcDistTable.merge(minMaxDist, on=['year_month', 'Store1'])
distTable

,Store1,Store2,year_month,measures,max,min
0,77,1,2018-06-01,0,8,0.0
1,77,2,2018-06-01,0,8,0.0
2,77,3,2018-06-01,1,8,0.0
3,77,4,2018-06-01,3,8,0.0
4,77,5,2018-06-01,0,8,0.0
...,...,...,...,...,...,...
2139,77,268,2019-01-01,1,100,0.0
2140,77,269,2019-01-01,71,100,0.0
2141,77,270,2019-01-01,82,100,0.0
2142,77,271,2019-01-01,57,100,0.0


In [397]:
distTable['magnitudeMeasure']= distTable.apply(lambda row: 1- (row['measures']-row['min'])/(row['max']-row['min']),axis=1)
distTable

,Store1,Store2,year_month,measures,max,min,magnitudeMeasure
0,77,1,2018-06-01,0,8,0.0,1.000
1,77,2,2018-06-01,0,8,0.0,1.000
2,77,3,2018-06-01,1,8,0.0,0.875
3,77,4,2018-06-01,3,8,0.0,0.625
4,77,5,2018-06-01,0,8,0.0,1.000
...,...,...,...,...,...,...,...
2139,77,268,2019-01-01,1,100,0.0,0.990
2140,77,269,2019-01-01,71,100,0.0,0.290
2141,77,270,2019-01-01,82,100,0.0,0.180
2142,77,271,2019-01-01,57,100,0.0,0.430


In [398]:
finalDistTable = distTable.groupby(['Store1','Store2'])['magnitudeMeasure'].mean().reset_index()
finalDistTable.columns = ['Store1','Store2','mag_measure']
finalDistTable

,Store1,Store2,mag_measure
0,77,1,0.950075
1,77,2,0.933215
2,77,3,0.406144
3,77,4,0.243139
4,77,5,0.541307
...,...,...,...
263,77,268,0.935928
264,77,269,0.417467
265,77,270,0.315052
266,77,271,0.488128


#### Merge nTotSals & nCustomers





In [400]:
corr_nSales = calculateCorrelation(inputTable=preTrialMeasures, metricCol='TOT_SALES',storeComparison='77')
corr_nSales

,Store1,Store2,corr_measure
0,77,1,0.841751
1,77,2,0.840647
2,77,3,0.967552
3,77,4,0.895463
4,77,5,0.935511
...,...,...,...
263,77,268,0.845355
264,77,269,0.889074
265,77,270,0.927951
266,77,271,0.931167


In [401]:
corr_nCustomers = calculateCorrelation(inputTable=preTrialMeasures, metricCol='nCustomers',storeComparison='77')
corr_nCustomers

,Store1,Store2,corr_measure
0,77,1,0.909962
1,77,2,0.881730
2,77,3,0.975036
3,77,4,0.916797
4,77,5,0.962518
...,...,...,...
263,77,268,0.913429
264,77,269,0.925431
265,77,270,0.895506
266,77,271,0.943397


In [406]:
magnitude_nSales = finalDistTable(inputTable=preTrialMeasures, metricCol='TOT_SALES',storeComparison='77')
magnitude_nSales

,Store1,Store2,mag_measure
0,77,1,0.938089
1,77,2,0.939103
2,77,3,0.417810
3,77,4,0.225170
4,77,5,0.609797
...,...,...,...
263,77,268,0.945110
264,77,269,0.516665
265,77,270,0.397266
266,77,271,0.535115


In [407]:
magnitude_nCustomers = finalDistTable(inputTable=preTrialMeasures, metricCol='nCustomers',storeComparison='77')
magnitude_nCustomers

,Store1,Store2,mag_measure
0,77,1,0.950075
1,77,2,0.933215
2,77,3,0.406144
3,77,4,0.243139
4,77,5,0.541307
...,...,...,...
263,77,268,0.935928
264,77,269,0.417467
265,77,270,0.315052
266,77,271,0.488128


#### Get control store




In [408]:
score_nSales = corr_nSales.merge(magnitude_nSales, on=['Store1','Store2'])
score_nSales['scoreNSales'] = score_nSales.apply(lambda row: row['corr_measure']*0.5 + row['mag_measure']*0.5, axis=1)
score_nSales = score_nSales.loc[:,['Store1','Store2', 'scoreNSales']]
score_nSales

,Store1,Store2,scoreNSales
0,77,1,0.889920
1,77,2,0.889875
2,77,3,0.692681
3,77,4,0.560317
4,77,5,0.772654
...,...,...,...
263,77,268,0.895233
264,77,269,0.702870
265,77,270,0.662609
266,77,271,0.733141


In [409]:
score_nCustomers = corr_nCustomers.merge(magnitude_nCustomers, on=['Store1','Store2'])
score_nCustomers['scoreNCust'] = score_nCustomers.apply(lambda row: row['corr_measure']*0.5 + row['mag_measure']*0.5, axis=1)
score_nCustomers = score_nCustomers.loc[:,['Store1','Store2','scoreNCust']]
score_nCustomers

,Store1,Store2,scoreNCust
0,77,1,0.930018
1,77,2,0.907473
2,77,3,0.690590
3,77,4,0.579968
4,77,5,0.751912
...,...,...,...
263,77,268,0.924679
264,77,269,0.671449
265,77,270,0.605279
266,77,271,0.715762


In [410]:
score_Control = score_nSales.merge(score_nCustomers, on=['Store1','Store2'])
score_Control

,Store1,Store2,scoreNSales,scoreNCust
0,77,1,0.889920,0.930018
1,77,2,0.889875,0.907473
2,77,3,0.692681,0.690590
3,77,4,0.560317,0.579968
4,77,5,0.772654,0.751912
...,...,...,...,...
263,77,268,0.895233,0.924679
264,77,269,0.702870,0.671449
265,77,270,0.662609,0.605279
266,77,271,0.733141,0.715762


In [411]:
score_Control['finalControlScore'] = score_Control.apply(lambda row: row['scoreNSales']*0.5 + row['scoreNCust']*0.5, axis=1)
score_Control

,Store1,Store2,scoreNSales,scoreNCust,finalControlScore
0,77,1,0.889920,0.930018,0.909969
1,77,2,0.889875,0.907473,0.898674
2,77,3,0.692681,0.690590,0.691635
3,77,4,0.560317,0.579968,0.570142
4,77,5,0.772654,0.751912,0.762283
...,...,...,...,...,...
263,77,268,0.895233,0.924679,0.909956
264,77,269,0.702870,0.671449,0.687159
265,77,270,0.662609,0.605279,0.633944
266,77,271,0.733141,0.715762,0.724452


In [412]:
final_control_store = score_Control['finalControlScore'].max()

In [413]:
score_Control[score_Control['finalControlScore']==final_control_store]

,Store1,Store2,scoreNSales,scoreNCust,finalControlScore
228,77,233,0.962765,0.981021,0.971893


#### Visualization the control store 

In [444]:
measureOverTime['Store_type'] = measureOverTime.apply(lambda row: 'Trail' if row['STORE_NBR']==77 else ('Control' if row['STORE_NBR']==233 else 'Other stores'), axis=1)
measureOverTime

,STORE_NBR,year_month,totSales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,Store_type
0,1,2018-06-01,4.8,2,1.000000,1.000000,2.400000,Other stores
1,1,2018-07-01,220.2,52,1.057692,1.181818,3.387692,Other stores
2,1,2018-08-01,171.8,41,1.000000,1.292683,3.241509,Other stores
3,1,2018-09-01,278.2,59,1.050847,1.209677,3.709333,Other stores
4,1,2018-10-01,186.4,44,1.022727,1.266667,3.270175,Other stores
...,...,...,...,...,...,...,...,...
3382,272,2019-02-01,395.5,45,1.066667,1.895833,4.346154,Other stores
3383,272,2019-03-01,478.5,52,1.115385,1.913793,4.310811,Other stores
3384,272,2019-04-01,415.5,50,1.040000,1.865385,4.283505,Other stores
3385,272,2019-05-01,322.8,36,1.138889,1.780488,4.421918,Other stores


In [421]:
measureOverTime['Store_type'].unique()

array(['Other stores', 'Trail', 'Control'], dtype=object)

In [424]:
measureOverTimeSales = measureOverTime.groupby(['year_month','Store_type'])['totSales'].mean().reset_index()
measureOverTimeSales

,year_month,Store_type,totSales
0,2018-06-01,Control,6.600000
1,2018-06-01,Other stores,24.654378
2,2018-06-01,Trail,15.600000
3,2018-07-01,Control,286.200000
4,2018-07-01,Other stores,623.767803
5,2018-07-01,Trail,289.100000
6,2018-08-01,Control,300.200000
7,2018-08-01,Other stores,601.354771
8,2018-08-01,Trail,247.600000
9,2018-09-01,Control,228.000000


In [425]:
measureOverTimeSales.set_index('year_month',inplace=True)

In [428]:
pastSales = measureOverTimeSales.loc['2018-06-01':'2019-01-01'].reset_index()
pastSales

,year_month,Store_type,totSales
0,2018-06-01,Control,6.600000
1,2018-06-01,Other stores,24.654378
2,2018-06-01,Trail,15.600000
3,2018-07-01,Control,286.200000
4,2018-07-01,Other stores,623.767803
5,2018-07-01,Trail,289.100000
6,2018-08-01,Control,300.200000
7,2018-08-01,Other stores,601.354771
8,2018-08-01,Trail,247.600000
9,2018-09-01,Control,228.000000


In [433]:
px.line(data_frame=pastSales, x='year_month', y='totSales', color='Store_type', title='Total sales by month',labels={'year_month':'Month of operation','totSales':'Total sales'})

In [439]:
measureOverTimeCusts = measureOverTime.groupby(['year_month','Store_type'])['nCustomers'].mean().reset_index()
measureOverTimeCusts

,year_month,Store_type,nCustomers
0,2018-06-01,Control,1.000000
1,2018-06-01,Other stores,3.304147
2,2018-06-01,Trail,2.000000
3,2018-07-01,Control,51.000000
4,2018-07-01,Other stores,70.640152
5,2018-07-01,Trail,50.000000
6,2018-08-01,Control,49.000000
7,2018-08-01,Other stores,71.034351
8,2018-08-01,Trail,45.000000
9,2018-09-01,Control,43.000000


In [440]:
measureOverTimeCusts.set_index('year_month',inplace=True)
pastCustomers = measureOverTimeCusts.loc['2018-06-01':'2019-01-01'].reset_index()
pastCustomers

,year_month,Store_type,nCustomers
0,2018-06-01,Control,1.000000
1,2018-06-01,Other stores,3.304147
2,2018-06-01,Trail,2.000000
3,2018-07-01,Control,51.000000
4,2018-07-01,Other stores,70.640152
5,2018-07-01,Trail,50.000000
6,2018-08-01,Control,49.000000
7,2018-08-01,Other stores,71.034351
8,2018-08-01,Trail,45.000000
9,2018-09-01,Control,43.000000


In [443]:
px.line(data_frame=pastCustomers, x='year_month', y='nCustomers', color='Store_type', title='Total customers by month',labels={'year_month':'Month of operation','nCustomers':'Total customers'})

## Assessment of trial